In [24]:
# Update package lists and install OpenCV with full dependencies
!apt-get update -y
!apt-get install -y libopencv-dev libopencv-core-dev libopencv-imgproc-dev libopencv-highgui-dev libopencv-imgcodecs-dev

# Verify OpenCV installation
import cv2
print("OpenCV version:", cv2.__version__)

# Create a directory for the project
!mkdir -p "C++ Project"

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [29]:


# Write the C++ code to a file
with open("C++ Project/circle_detection.cpp", "w") as f:
    f.write("""
#include <opencv2/opencv.hpp>
#include <iostream>
#include <vector>
#include <cmath>

using namespace cv;
using namespace std;

int main(int argc, char** argv) {
    // Load the image
    Mat image = imread(argv[1], IMREAD_COLOR);
    if (image.empty()) {
        cout << "Could not open or find the image" << endl;
        return -1;
    }

    // Convert to grayscale for edge-based detection as primary method
    Mat gray;
    cvtColor(image, gray, COLOR_BGR2GRAY);
    Mat blurred;
    GaussianBlur(gray, blurred, Size(5, 5), 1, 1);

    // Use adaptive thresholding to highlight ring boundaries
    Mat thresh;
    adaptiveThreshold(blurred, thresh, 255, ADAPTIVE_THRESH_GAUSSIAN_C, THRESH_BINARY_INV, 11, 2);
    imwrite("thresh.jpg", thresh);

    // Apply minimal morphological operations to enhance edges
    Mat kernel = getStructuringElement(MORPH_ELLIPSE, Size(3, 3));
    morphologyEx(thresh, thresh, MORPH_CLOSE, kernel, Point(-1, -1), 1);
    imwrite("processed_thresh.jpg", thresh);

    // Find contours
    vector<vector<Point>> contours;
    vector<Vec4i> hierarchy;
    findContours(thresh, contours, hierarchy, RETR_CCOMP, CHAIN_APPROX_SIMPLE);
    cout << "Number of contours after thresholding: " << contours.size() << endl;

    // Draw all contours for debugging
    Mat contourImage = image.clone();
    for (size_t i = 0; i < contours.size(); i++) {
        drawContours(contourImage, contours, (int)i, Scalar(0, 0, 255), 2);
    }
    imwrite("contours.jpg", contourImage);

    // Estimate the center using the largest contour
    Point2f center(image.cols / 2, image.rows / 2);
    double maxArea = 0;
    for (size_t i = 0; i < contours.size(); i++) {
        double area = contourArea(contours[i]);
        if (area > maxArea && hierarchy[i][3] == -1) {
            maxArea = area;
            Moments m = moments(contours[i]);
            if (m.m00 != 0) {
                center = Point2f(m.m10 / m.m00, m.m01 / m.m00);
            }
        }
    }
    cout << "Estimated center: (" << center.x << ", " << center.y << ")" << endl;

    // Fit circles with very relaxed criteria
    vector<Vec3f> circles;
    for (size_t i = 0; i < contours.size(); i++) {
        if (contours[i].size() < 5) continue;
        double area = contourArea(contours[i]);
        if (area < 50) continue; // Extremely relaxed area threshold

        double perimeter = arcLength(contours[i], true);
        double circularity = 4 * CV_PI * area / (perimeter * perimeter);
        if (circularity < 0.3) continue; // Extremely relaxed circularity threshold

        Point2f circleCenter;
        float radius;
        minEnclosingCircle(contours[i], circleCenter, radius);
        double dist = sqrt(pow(circleCenter.x - center.x, 2) + pow(circleCenter.y - center.y, 2));
        cout << "Contour " << i << ": area=" << area << ", circularity=" << circularity
             << ", radius=" << radius << ", dist from center=" << dist << endl;

        if (dist < 300 && radius > 2) { // Extremely relaxed distance and radius thresholds
            circles.push_back(Vec3f(circleCenter.x, circleCenter.y, radius));
        }
    }
    cout << "Number of detected circles: " << circles.size() << endl;

    // Sort circles by radius (largest first) and enforce non-crossing
    vector<Vec3f> concentricCircles;
    if (!circles.empty()) {
        sort(circles.begin(), circles.end(), [](const Vec3f& a, const Vec3f& b) {
            return a[2] > b[2];
        });

        concentricCircles.push_back(circles[0]);
        int minSpacing = 2; // Extremely relaxed spacing
        for (size_t i = 1; i < circles.size(); i++) {
            Vec3f curr = circles[i];
            bool isConcentric = true;
            for (const Vec3f& prev : concentricCircles) {
                double dist = sqrt(pow(curr[0] - prev[0], 2) + pow(curr[1] - prev[1], 2));
                if (dist > 300 || curr[2] >= prev[2] - minSpacing) {
                    isConcentric = false;
                    break;
                }
            }
            if (isConcentric && concentricCircles.size() < 7) {
                concentricCircles.push_back(curr);
            }
        }

        // Ensure exactly 7 circles if possible
        while (concentricCircles.size() < 7 && !circles.empty()) {
            Vec3f last = concentricCircles.empty() ? circles[0] : concentricCircles.back();
            int newRadius = last[2] - minSpacing * 2;
            if (newRadius > 2) {
                concentricCircles.push_back(Vec3f(last[0], last[1], newRadius));
            } else {
                break;
            }
        }
        if (concentricCircles.size() > 7) {
            concentricCircles.resize(7);
        }
    }

    // Draw and label the 7 concentric circles with green lines
    for (size_t i = 0; i < concentricCircles.size(); i++) {
        Vec3f c = concentricCircles[i];
        Point centerPoint = Point(cvRound(c[0]), cvRound(c[1]));
        int radius = cvRound(c[2]);
        circle(image, centerPoint, radius, Scalar(0, 255, 0), 4, LINE_AA); // Green circle
        putText(image, to_string(i + 1), centerPoint, FONT_HERSHEY_SIMPLEX, 1, Scalar(0, 255, 0), 2); // Label with number
    }

    // Save the result
    imwrite("output.jpg", image);

    return 0;
}
""")

# Compile the C++ code with explicit OpenCV flags
import subprocess
try:
    result = subprocess.run(['g++', '-I/usr/include/opencv4', 'C++ Project/circle_detection.cpp', '-o', 'C++ Project/circle_detection',
                            '-L/usr/lib', '-lopencv_core', '-lopencv_imgproc', '-lopencv_highgui', '-lopencv_imgcodecs'],
                            capture_output=True, text=True, timeout=30)
    print("Compilation output:", result.stdout)
    print("Compilation errors (if any):", result.stderr)
except subprocess.TimeoutExpired:
    print("Compilation timed out")
except Exception as e:
    print("Compilation failed:", str(e))

# Upload the image
from google.colab import files
try:
    uploaded = files.upload()
    if not uploaded:
        raise ValueError("No file uploaded")
    image_filename = list(uploaded.keys())[0]
except Exception as e:
    print("Image upload failed:", str(e))
    raise

# Run the compiled program with the uploaded image
!chmod +x "C++ Project/circle_detection"
try:
    result = subprocess.run(['C++ Project/circle_detection', image_filename], capture_output=True, text=True, timeout=30)
    print("Program output:", result.stdout)
    print("Program errors (if any):", result.stderr)
except subprocess.TimeoutExpired:
    print("Program execution timed out")
except Exception as e:
    print("Program execution failed:", str(e))

# Check if output and debug images exist and display them
import os
import cv2
import matplotlib.pyplot as plt

if os.path.exists("output.jpg"):
    output_image = cv2.imread("output.jpg")
    if output_image is None:
        print("Failed to load output.jpg")
    else:
        output_image_rgb = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 10))
        plt.imshow(output_image_rgb)
        plt.title("Output Image")
        plt.axis('off')
        plt.show()
else:
    print("Error: Output image 'output.jpg' was not generated.")

# Display debug images if they exist
for debug_file in ["thresh.jpg", "processed_thresh.jpg", "contours.jpg"]:
    if os.path.exists(debug_file):
        debug_image = cv2.imread(debug_file)
        if debug_image is None:
            print(f"Failed to load {debug_file}")
        else:
            debug_image_rgb = cv2.cvtColor(debug_image, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(10, 10))
            plt.imshow(debug_image_rgb)
            plt.title(debug_file)
            plt.axis('off')
            plt.show()
    else:
        print(f"Debug file {debug_file} was not generated.")

Output hidden; open in https://colab.research.google.com to view.